# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
%pip install azureml-train-automl-runtime==1.57.0 -q
%pip install --upgrade azureml-sdk[notebooks,automl] -q
%pip install azureml-widgets -q

StatementMeta(8d137bca-db0d-4176-840a-7e7df7eff5a6, 1, 12, Finished, Available, Finished)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
jupyter-client 7.4.4 requires tornado>=6.2, but you have tornado 6.1 which is incompatible.
azureml-opendatasets 1.47.0 requires azureml-core~=1.47.0, but you have azureml-core 1.57.0.post3 which is incompatible.
azureml-opendatasets 1.47.0 requires azureml-dataset-runtime[fuse,pandas]~=1.47.0, but you have azureml-dataset-runtime 1.57.0 which is incompatible.
azureml-opendatasets 1.47.0 requires azureml-telemetry~=1.47.0, but you have azureml-telemetry 1.57.0 which is incompatible.
You should consider upgrading via the '/nfs4/pyenv-7cda8f43-09e4-4353-94c3-b37a3b0bd095/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolve

In [2]:


import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

StatementMeta(8d137bca-db0d-4176-840a-7e7df7eff5a6, 1, 18, Finished, Available, Finished)

SDK version: 1.58.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
session_id = "268818"
subscription_id= "d7f39349-a66b-446e-aba6-0053c2cf1c11"
resource_group= f"aml-quickstarts-{session_id}"
workspace_name= f"quick-starts-ws-{session_id}"

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'automl-exp'
project_folder = './capstone'
experiment=Experiment(ws, experiment_name)
experiment

StatementMeta(8d137bca-db0d-4176-840a-7e7df7eff5a6, 1, 19, Finished, Available, Finished)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code GCKW9E6T8 to authenticate.
Interactive authentication successfully completed.
quick-starts-ws-268818
aml-quickstarts-268818
westeurope
d7f39349-a66b-446e-aba6-0053c2cf1c11


Name,Workspace,Report Page,Docs Page
automl-exp,quick-starts-ws-268818,Link to Azure Machine Learning studio,Link to Documentation


In [6]:


# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "heart-failure"
description_text = "Heart Failure Prediction Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/hknguyen20/nd00333-capstone/refs/heads/master/heart.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)
df = dataset.to_pandas_dataframe()
df.head()


StatementMeta(8d137bca-db0d-4176-840a-7e7df7eff5a6, 1, 22, Finished, Available, Finished)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': 'ba91beef-4278-4a82-a99e-a8f22a4c5ee8'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': 'ba91beef-4278-4a82-a99e-a8f22a4c5ee8', 'run_id': 'ba91beef-4278-4a82-a99e-a8f22a4c5ee8'}


,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,False,Normal,172,False,0.0,Up,False
1,49,F,NAP,160,180,False,Normal,156,False,1.0,Flat,True
2,37,M,ATA,130,283,False,ST,98,False,0.0,Up,False
3,48,F,ASY,138,214,False,Normal,108,True,1.5,Flat,True
4,54,M,NAP,150,195,False,Normal,122,False,0.0,Up,False


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-cluster-1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_v2',# for GPU, use "STANDARD_NC6"
                                                           vm_priority = 'lowpriority', # optional
                                                           min_nodes=1,
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

StatementMeta(8d137bca-db0d-4176-840a-7e7df7eff5a6, 1, 23, Submitted, Running, Running)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.

In [10]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 40,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted',
    "enable_early_stopping": True,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="HeartDisease",   
                             path = project_folder,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

StatementMeta(8d137bca-db0d-4176-840a-7e7df7eff5a6, 1, 26, Finished, Available, Finished)

In [11]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config)

StatementMeta(8d137bca-db0d-4176-840a-7e7df7eff5a6, 1, 27, Finished, Available, Finished)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-exp,AutoML_0b035984-dce8-4b10-8a54-88f04cf31c41,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
from azureml.widgets import RunDetails
RunDetails(automl_run).show()
automl_run.wait_for_completion()

StatementMeta(8d137bca-db0d-4176-840a-7e7df7eff5a6, 1, 28, Submitted, Running, Running)

Initializing logging file for interpret-community


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_0b035984-dce8-4b10-8a54-88f04cf31c41',
 'target': 'compute-cluster-1',
 'status': 'Completed',
 'startTimeUtc': '2024-10-18T07:57:51.37449Z',
 'endTimeUtc': '2024-10-18T08:54:22.438331Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'compute-cluster-1',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-exp","subscription_id":"d7f39349-a66b-446e-aba6-0053c2cf1c11","resource_group":"aml-quickstarts-268818","workspace_name":"quick-starts-ws-268818","region":"westeurope","compute_target":"compute

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_run, fitted_model = automl_run.get_output()
print(best_run)

StatementMeta(8d137bca-db0d-4176-840a-7e7df7eff5a6, 1, 29, Finished, Available, Finished)

[08:54:38] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:1203: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[08:54:38] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:888: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[08:54:38] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, pl

In [14]:
#TODO: Save the best model
best_run_metrics = best_run.get_metrics()
best_model = best_run.register_model(model_path='./outputs/', model_name='best_model.pkl',
                        properties=best_run_metrics)

print(best_model)

StatementMeta(8d137bca-db0d-4176-840a-7e7df7eff5a6, 1, 30, Finished, Available, Finished)

Model(workspace=Workspace.create(name='quick-starts-ws-268818', subscription_id='d7f39349-a66b-446e-aba6-0053c2cf1c11', resource_group='aml-quickstarts-268818'), name=best_model.pkl, id=best_model.pkl:1, version=1, tags={}, properties={'log_loss': '0.3444276906705273', 'average_precision_score_micro': '0.9399465971081306', 'AUC_macro': '0.9418521867445374', 'weighted_accuracy': '0.8827210476365683', 'AUC_weighted': '0.9418521867445373', 'recall_score_macro': '0.8730804436241554', 'recall_score_weighted': '0.878057811753464', 'norm_macro_recall': '0.746160887248311', 'precision_score_weighted': '0.8796926683877683', 'matthews_correlation': '0.7529010724512283', 'accuracy': '0.878057811753464', 'average_precision_score_macro': '0.9414033028433888', 'AUC_micro': '0.9412813760586618', 'precision_score_macro': '0.8798786736090314', 'average_precision_score_weighted': '0.9425767075521012', 'f1_score_micro': '0.878057811753464', 'recall_score_micro': '0.878057811753464', 'f1_score_weighted': 

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
best_run.download_file('./outputs/scoring_file_v_1_0_0.py', 'scoring.py')
best_run.download_file('./outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
